## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [13]:
import glob

import numpy as np
import scipy as sp
import scipy.io

import scipy.signal
from scipy.signal import butter, lfilter

import collections
import matplotlib.pyplot as plt


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], [] 
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
  
    # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandPassFilter(data, lowcut= 0.67, highcut =5, fs=125, order=5):
    """
    Create a bandpass filter

    Args:
        data: a numpy array of the signal needed to be filtered.
        lowcut: low cut off frequency.
        highcut: high cut off frequency.
        fs: sampling rate
        order: The order of the filter

    Returns:
        filtered signal
    """
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_bandpass(lowcut=0.67, highcut=5, fs=125, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def FreqVarification(Nprev, Ncrrent):
    """
    Varify that the heart rate between to successive time frames does not exceed 11 BPM

    Args:
        Nprev: the index of the dominant previous time frame PPG frequency in the signal frequency NumPy array.
        Nprev: the index of the dominant current time frame PPG frequency in the frequency NumPy array.

    Returns:
        The index of the dominant current time frame PPG frequency after the varification
    """
    # maximum difference indexes between two dominant frequences in two successive frames
    ceta = 6
    # value used to increase or decrease the index of the current dominant frequency if it exceeds 11 BPM
    taw = 2

    if Ncrrent - Nprev >= ceta:
        return Nprev +taw
    if Ncrrent - Nprev <= -1 * ceta:
        return Nprev - taw
    return Ncrrent

def GenerateFramesForData (data, ref, fs, window_length_s, window_shift_s):

    """
    Splits the signals into 8 seconds frames with 2 seconds shift between them and filters it 

    Args:
        data: NumPy array of all signals in a file.
        ref: NumPy array of the true heart rate value for every 8 seconds.
        fs: sampling rate
        window_length_s: Window length in seconds 
        window_shift_s: shift between windows in seconds

    Returns:
        features: NumPy array has all filtered signals in frames
        labels: NumPy array for the heart rate for the corresponding frame in the feature array
    """
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    labels, subjects, activities, features = [],[],[],[]
    
    # variable for storing number of frames in the file 
    k= 0
    # genetating frames
    for i in range (0, len(data[0])-window_length, window_shift):
        window = range (i, i+window_length)
        #ECG = data[0][window]
        #ECG = BandPassFilter(ECG)

        #PPG_1 = data[1][window]
        #PPG_2 = data[2][window]
        PPG = data[0][window]
        PPG = BandPassFilter(PPG)

        x = data[1][window]
        x = BandPassFilter(x)

        y = data[2][window]
        y = BandPassFilter(y)

        z = data[3][window]
        z = BandPassFilter(z)

        # Calculate the magnitude acceleration
        magAcc = np.sum(np.square(np.vstack((x,y,z))), axis= 0)
        magAcc = BandPassFilter(magAcc)
        features.append(( PPG, x, y, z, magAcc))
        k +=1

    labels = ref[0:k]
    features = np.array(features)
        
    return features, labels

def getFirstDomFreq(features,label, fs=125):

    """
    Calculate the dominant frequency in the first time frame.

    Args:
        features: NumPy array has all filtered signals in frames
        labels: NumPy array for the heart rate for the corresponding frame in the feature array
        fs: sampling rate

    Returns:
        the dominant frequency of the first frame for every subject
        
    """
    # get the desired features 
    PPG = features[0]
    
    # FFT
    fft_len = max(len(PPG),4096)
    fftFreqs = np.fft.rfftfreq(fft_len, 1/fs)
    fft_PPG = np.abs(np.fft.rfft(PPG, fft_len))
    
    #Calculate dominate freqs
    freqs = fftFreqs[(fftFreqs>0.67)&(fftFreqs<5)]
    dominantFreq_PPG = freqs[np.argmax(fft_PPG[(fftFreqs>0.67)&(fftFreqs<5)])]

    # get the fundamintal frequency if the dominant frequency is equal to the first harmonic frequency.
    # the first harmonic frequency will be bigger that 140 BPM (2.345 HZ)
    firstHarmonicCheckValue =  2.345
    if  dominantFreq_PPG > firstHarmonicCheckValue:
       
        return dominantFreq_PPG/2
      
    return dominantFreq_PPG

def getErrorConfidancePerwindonw(features,label, prev_dominantFreq_PPG, fs=125):

    """
    Calculate absolute error of the heart rate for every frame and its confidance value.

    Args:
        features: NumPy array has all filtered signals in frames
        labels: NumPy array for the heart rate for the corresponding frame in the feature array
        fs: sampling rate

    Returns:
        the dominant frequency of the first frame for every subject
        absError: the absolute error of heart beat rate for every frame
        confidence: value that represents how the algorithm confidant on heat rate estimate
        dominantFreq_PPG: the dominant frequency of the current frame        
    """

    # get the desired features 
    PPG = features[0]
    X = features[1]
    Y = features[2]
    Z = features[3]
    mag = features[4]

    # FFT
    fft_len = max(len(PPG),4096)
    fftFreqs = np.fft.rfftfreq(fft_len, 1/fs)
    freqs = fftFreqs[(fftFreqs>0.67)&(fftFreqs<5)]

    fft_PPG = np.abs(np.fft.rfft(PPG, fft_len))
    fft_X = np.abs(np.fft.rfft(X, fft_len))
    fft_Y = np.abs(np.fft.rfft(Y, fft_len))
    fft_Z = np.abs(np.fft.rfft(Z, fft_len))
    fft_mag = np.abs(np.fft.rfft(mag, fft_len))

    fft_X_wind = fft_X[(fftFreqs>0.67)&(fftFreqs<5)]
    fft_Y_wind = fft_Y[(fftFreqs>0.67)&(fftFreqs<5)]
    fft_Z_wind = fft_Z[(fftFreqs>0.67)&(fftFreqs<5)] 
    fft_mag_wind = fft_mag[(fftFreqs>0.67)&(fftFreqs<5)]

    # Calculate the dominant frequences for x acceleration
    threshold_X = 0.5* np.max(fft_X_wind)
    F_dominant_X = freqs[fft_X_wind > threshold_X]
    M_dominant_X = fft_X_wind [fft_X_wind > threshold_X]

    # Calculate the dominant frequences for y acceleration
    threshold_Y = 0.5* np.max(fft_Y_wind)
    F_dominant_Y = freqs[fft_Y_wind > threshold_Y]
    M_dominant_Y = fft_Y_wind [fft_Y_wind > threshold_Y]

    # Calculate the dominant frequences for z acceleration    
    threshold_Z = 0.5* np.max(fft_Z_wind)
    F_dominant_Z = freqs[fft_Z_wind > threshold_Z]
    M_dominant_Z = fft_Z_wind [fft_Z_wind > threshold_Z]
    
    # Calculate the dominant frequences for mag acceleration    
    threshold_mag = 0.5* np.max(fft_mag_wind)
    F_dominant_mag = freqs[fft_mag_wind > threshold_mag]
    M_dominant_mag = fft_mag_wind [fft_mag_wind > threshold_mag]

    # form one array for all dominant acceleration of all channels
    dominant_ACC = np.append(F_dominant_X, F_dominant_Y) 
    dominant_ACC = np.append(dominant_ACC, F_dominant_Z)
    dominant_ACC = np.append(dominant_ACC, F_dominant_mag)
    dominant_ACC = np.unique(dominant_ACC)
    

    # Remove frequences in dominant_ACC array that equal to the 
    # previous heart rate dominant frequences +-0.2 HZ
    prev_freq_tolerance = 0.2
    for d in dominant_ACC:
        if d > prev_dominantFreq_PPG - prev_freq_tolerance and d < prev_dominantFreq_PPG + prev_freq_tolerance:
            dominant_ACC = np.delete(dominant_ACC, np.argwhere(dominant_ACC == d))

    # Filter signal with in 40 to 240 BPM by removing the acceleration dominant frequences from it 
    for f in fftFreqs:
        if f in dominant_ACC:
            fft_PPG[np.argwhere(fftFreqs==f)] = 0.0   
  
    
    #Calculate dominate freqs
    dominantFreq_PPG = freqs[np.argmax(fft_PPG[(fftFreqs>0.67)&(fftFreqs<5)])]

    # varify that the heart rate difference between two successive frames not more than 11 BPM    
    indx_dominantFreq_PPG = FreqVarification(np.argwhere(freqs==prev_dominantFreq_PPG), np.argwhere(freqs==dominantFreq_PPG))

    dominantFreq_PPG = freqs[indx_dominantFreq_PPG][0][0]

   
    # calculate the energy around the expexted frequency (Confidance)
    spectral_energy_PPG = (fft_PPG)
    band_around_dominant_freq = 0.15
    dom_wind = (fftFreqs>(dominantFreq_PPG- band_around_dominant_freq)) & (fftFreqs< (dominantFreq_PPG+ band_around_dominant_freq))
    
    confidence = np.sum(spectral_energy_PPG[dom_wind])/np.sum(spectral_energy_PPG)

    absError = abs ((dominantFreq_PPG*60)- label)
    
    return absError, confidence, dominantFreq_PPG

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using scipy. "scipy.io.loadmat(f)"

    data = LoadTroikaDataFile(data_fl)
    ref = scipy.io.loadmat(ref_fl).get('BPM0')
    features, labels = GenerateFramesForData(data, ref, 125, 8, 2)

    absErrorS = []
    ConfidanceS = []
    dominantFreq_PPG = getFirstDomFreq(features[0], labels[0])
    imageIndx = 0
    for (f, l) in zip(features, labels):
        absError, confidence, dominantFreq_PPG = getErrorConfidancePerwindonw(f, l,dominantFreq_PPG)
        absErrorS = np.append(absErrorS, absError)
        ConfidanceS = np.append(ConfidanceS, confidence)
        imageIndx +=1
        
    # Compute pulse rate estimates and estimation confidence.

    # Return per-estimate absolute error and confidence as a 2-tuple of numpy arrays.
    return absErrorS, ConfidanceS

In [14]:
Evaluate()

6.8336051962800388

> - ### **To run this code** 
    - You need to import
        - [glob](https://docs.python.org/3/library/glob.html)
        - [numpy](https://numpy.org/)
        - [scipy](https://www.scipy.org/)
        - collections
        - matplotlib
    - The notebook consists of two main code cells. 
    - The first cell contains all the algorithm functions. 
    - The second one contains the Evaluate method which kicks off the algorithm.
     
> - ### **Data Description** 
Dataset Troika[1] was used to build the algorithm. You can find the dataset under datasets/troika/training_data. 
    - What activities were performed in the dataset.
        - The dataset contains (ECG signal, PPG signal, (x,y,z) accelerattion signal) 
        - The ground truth of the hear beat was obtained from the ECG signal
        - The magnitude acceleration was obtained from (x,y,z) acceleration signals
        - All signals are filtered by a band bass filter 40:240 BPM
    - Features of the sensor that was used.
        - PPG, (x,y,z) accelerattion.  
    - The quantity of data in the dataset.
        - The dataset consists of 24 files.
        - Files with similar name "DATA_01_TYPE01" contain a variable 'sig'. It ehas 6 rows. The first row is a simultaneous recording of ECG. The second row and the third row are two channels of PPG. The last three rows are simultaneous recordings of acceleration data (in x-, y-, and z-axis).
        - Files with a similar name "REF_01_TYPE01" contain a variable 'BPM0', which holds the ground-truth heart rate.
        - All signals were sampled at 125 Hz.
    - Short-comings of the dataset.
        - The length of some "REF_01_TYPE01" files are not equal to the length of total 8 seconds frames in the "DATA_01_TYPE01" files. 
    
    
> - ### **Algorithhm Description**
>   - #### **How the algorithm works**
>        - The algorithm loads the database into two NumPy arrays. One for data and another for references.
>        - For every file in the database, the data is divided into time frames. Every frame consists of 8 seconds.
>        - These frames are stored in a NumPy array called features. 
>        - The true heartbeat rate for every frame is stored in a NumPy array called labels.
>        - All feature signals are filtered by a bandpass filter (40:240) BPM.
>        - The algorithm starts by converting the frame signal from the time domain to the frequency domain. 
>        - For every time window, FFT is used to calculate the spectrum of each channel of acceleration data and PPG signal, from which dominant frequencies are determined. 
>        - The dominant frequencies of the acceleration are the ones corresponding to the spectral peaks with an amplitude larger than 50% of the maximum amplitude in a given spectrum.
>        - The dominant frequencies of the acceleration are removed from the PPG signal.
>        - The dominant frequencies of the acceleration may have values that equal to the PPG dominant frequency. To prevent removing the dominant frequency of the PPG signal, the dominant frequency of the previous frame was used to remove any dominant acceleration frequencies that equal it (+- 12 BPM). 
>        - The heartbeat rate was obtained from the dominant frequency of the PPG signal
>        - The algorithm contains a method for heartbeat rate verification. This method prevents a large change in the estimated BPM values in two successive time windows. The change of BPM values in two successive time windows was prevented to exceed 10 BPM.
>   - #### **The specific aspects of the physiology of heartbeat rate**
      - The heartbeat rate is always between 40:240 BPM, so a bandpass filter was used to remove frequencies out of this range. 
>        - The change of BPM values in two successive time windows rarely exceeds 10 BPM, so a heartbeat rate verification method was designed based on this concept.  
>   - #### **Confidence rate**
      - The energy concentration in the frequency spectrum around the estimated pulse rate is used to obtain a confidence rate for every estimate. This was done by summing frequency spectrum near the pulse rate estimate and dividing it by the sum of the entire spectrum. A high confidence rate means better estimations because it means that the signal has low noise, and there is one main dominant frequency in the signal.  
>   - #### **Algorithm outputs** 
>     - The mean absolute error of the best 90% of heartbeat rate estimates for the entire dataset.
>   - #### **caveats on algorithm outputs**
>     - The algorithm generates output every 2 seconds. 
>   - #### **common failure modes**
>     - The common failure takes place when the heartbeat of the first time frame was set wrong. This could happen if there is a strong acceleration signal in the first time frame. To avoid that, the subject is required to hold his hand for the first 8 seconds. 
>     - When there is no contact between the PPG sensor and the subject's skin.

>- #### **Algorithm Performance**
    - The mean absolute error (MAE) of the best 90 estimates was used to calculate the algorithm performance, and it was 6.7 through the entire dataset.
    - The algorithm was tested on another dataset, and the (MAE) of the best 90 estimates was 6.83
    - The energy concentration in the frequency spectrum around the estimated pulse rate is used to obtain a confidence rate for every estimate. This was done by summing frequency spectrum near the pulse rate estimate and dividing it by the sum of the entire spectrum.

> -  ### Citations
>       1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 